In [4]:
# Fix newton_krylov call by removing unsupported tol argument
import numpy as np
from scipy.optimize import newton_krylov
import matplotlib.pyplot as plt

# Parameters remain the same: B, mu, C, dx, dt, Nx, Nt, x, initial h...

# ... (reuse spatial_rhs) ...

def implicit_cn_step(h_old):
    rhs_old = spatial_rhs(h_old)
    def residual(h_new):
        return h_new - h_old - dt/2 * (rhs_old + spatial_rhs(h_new))
    # solve nonlinear system: remove tol, rely on default
    h_new = newton_krylov(residual, h_old, method='lgmres', verbose=False)
    return h_new

# (Continue time-stepping and diagnostics as before)

import numpy as np
from scipy.optimize import newton_krylov
import matplotlib.pyplot as plt

# Parameters
B, mu = 1.0, 1.0
C = B / (12 * mu)
dx = 1.0
dt = 0.1 * dx**6 / C  # time step
Nx = 201
Nt = 200

# Spatial grid and initial condition (Gaussian)
x = np.linspace(-50, 50, Nx)
h = np.exp(-x**2 / 10)
h /= np.sum(h) * dx  # normalize total mass

# Helper: compute fifth derivative flux divergence
def spatial_rhs(h_field):
    # 4th derivative
    p = (np.roll(h_field, -2) - 4*np.roll(h_field, -1) + 6*h_field - 4*np.roll(h_field, 1) + np.roll(h_field, 2)) / dx**4
    # flux at half-points
    q_ip = C * 0.5 * (h_field**3 + np.roll(h_field, -1)**3) * (np.roll(p, -1) - p) / dx
    q_im = C * 0.5 * (np.roll(h_field, 1)**3 + h_field**3) * (p - np.roll(p, 1)) / dx
    # divergence
    return -(q_ip - q_im) / dx

def implicit_cn_step(h_old):
    rhs_old = spatial_rhs(h_old)
    def residual(h_new):
        return h_new - h_old - dt/2 * (rhs_old + spatial_rhs(h_new))
    # solve nonlinear system with Jacobian-free Newton-Krylov
    h_new = newton_krylov(residual, h_old, method='lgmres', verbose=False, tol=1e-8)
    return h_new

# Time-stepping and diagnostics
times, widths, amps = [], [], []
for n in range(1, Nt+1):
    h = implicit_cn_step(h)
    if n % 20 == 0:
        t = n * dt
        mass = np.sum(h) * dx
        width = np.sqrt(np.sum(h * x**2) * dx / mass)
        amp = np.max(h)
        times.append(t)
        widths.append(width)
        amps.append(amp)

# Fit self-similar exponents
beta, _ = np.polyfit(np.log(times), np.log(widths), 1)
alpha, _ = np.polyfit(np.log(times), np.log(amps), 1)
print(f"Fully implicit CN exponents: β ≈ {beta:.3f}, α ≈ {alpha:.3f}")

# Plot results
plt.figure()
plt.loglog(times, widths, 'o-', label=f'β={beta:.3f}')
plt.loglog(times, amps, 's-', label=f'α={alpha:.3f}')
plt.xlabel('t')
plt.ylabel('scaling')
plt.legend()
plt.show()



ValueError: Unknown parameter tol